# [실습] CNN활용_전이학습_InceptionV3_transfer_learning

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


#### 데이터 수집

In [3]:
# MNIST 데이터셋을 로드하고, 정규화한다
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train[:10000] / 255.0, x_test[:2000] / 255.0
#x_train, x_test = x_train[:3] / 255.0, x_test[:2000] / 255.0
## 훈련데이터는 10000개, 검증 데이터는 2000개만 사용
y_train, y_test = y_train[:10000], y_test[:2000]

print(x_train.shape)

(10000, 28, 28)


#### 사전 학습된 Inceptionv3 모델 살펴보기

In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras import layers

from tensorflow.keras.optimizers import RMSprop

### 구글 Inception V3를 활용한 손글씨 MNIST 분류(ver1.0)

In [5]:
model = InceptionV3()
model.summary()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-02-23 15:56:11.443552: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-23 15:56:11.443640: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [6]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True,
            show_layer_names=True, to_file= 'inceptionV3.png')

In [7]:
# tensorflow가 있는 가상환경에서 
# conda install scikt-image

In [9]:
from skimage.transform import resize

## 구글 Inception V3를 활용한 손글씨 MNIST 분류(ver2.0)

In [11]:
x_train_preprocess = np.zeros((x_train.shape[0], 75, 75, 3), 
                              dtype=np.float32)
print(x_train_preprocess.shape)

(10000, 75, 75, 3)


In [12]:
for i, img in enumerate(x_train):
    img_resize = resize(img, (75, 75), anti_aliasing=True)
    x_train_preprocess[i] = np.dstack([img_resize, img_resize, img_resize])

x_test_preprocess = np.zeros((x_test.shape[0], 75, 75, 3), 
                             dtype=np.float32)

print(x_test_preprocess.shape)

(2000, 75, 75, 3)


In [13]:
# 검증 데이터 각각의 크기를 75x75로 변형 (인셉션모델 최소 입력 크기)
for i, img in enumerate(x_test):
    img_resize = resize(img, (75, 75), anti_aliasing=True)
    x_test_preprocess[i] = np.dstack([img_resize, img_resize, img_resize])

In [16]:
pre_trained_model = InceptionV3(input_shape=(75,75,3), include_top=False)

In [17]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [18]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

x1 = layers.Flatten()(last_output)
x2 = layers.Dense(64, activation='relu')(x1)
x3 = layers.Dense(10, activation='softmax')(x2)

model = Model(pre_trained_model.input, x3)

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics='acc')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 75, 75, 3)]  0           []                               
                                                                                                  
 conv2d_188 (Conv2D)            (None, 37, 37, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_188 (Batch  (None, 37, 37, 32)  96          ['conv2d_188[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_188 (Activation)    (None, 37, 37, 32)   0           ['batch_normalization_188[0][

In [20]:
history = model.fit(x_train_preprocess, y_train, 
                    epochs=20,
                    validation_data=(x_test_preprocess, y_test))

Epoch 1/20


2023-02-23 16:11:19.247795: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-23 16:11:20.659477: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - ETA: 0s - loss: 0.2461 - acc: 0.9306

2023-02-23 16:11:32.522347: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 16s 35ms/step - loss: 0.2461 - acc: 0.9306 - val_loss: 0.1328 - val_acc: 0.9600
Epoch 2/20
313/313 [==============================] - 9s 29ms/step - loss: 0.0721 - acc: 0.9769 - val_loss: 0.0944 - val_acc: 0.9695
Epoch 3/20
313/313 [==============================] - 9s 28ms/step - loss: 0.0435 - acc: 0.9868 - val_loss: 0.0962 - val_acc: 0.9720
Epoch 4/20
313/313 [==============================] - 9s 28ms/step - loss: 0.0290 - acc: 0.9906 - val_loss: 0.0850 - val_acc: 0.9730
Epoch 5/20
313/313 [==============================] - 9s 28ms/step - loss: 0.0215 - acc: 0.9937 - val_loss: 0.0981 - val_acc: 0.9695
Epoch 6/20
313/313 [==============================] - 9s 29ms/step - loss: 0.0136 - acc: 0.9962 - val_loss: 0.0776 - val_acc: 0.9735
Epoch 7/20
313/313 [==============================] - 9s 29ms/step - loss: 0.0101 - acc: 0.9976 - val_loss: 0.0800 - val_acc: 0.9755
Epoch 8/20
313/313 [==============================] - 9s 29ms/step - loss: 0.01

In [21]:
# 학습을 마치면 모델을 저장하는 습관을 들이기를 추천합니다.
model.save('252.1_InceptionV3.h5')

In [22]:
import pandas as pd
pd.DataFrame(history.history)

,loss,acc,val_loss,val_acc
0,0.246058,0.9306,0.132790,0.9600
1,0.072102,0.9769,0.094387,0.9695
2,0.043464,0.9868,0.096224,0.9720
3,0.029004,0.9906,0.084992,0.9730
4,0.021523,0.9937,0.098064,0.9695
5,0.013618,0.9962,0.077611,0.9735
6,0.010122,0.9976,0.079952,0.9755
7,0.012728,0.9962,0.092948,0.9730
8,0.011342,0.9968,0.091806,0.9770
9,0.017725,0.9945,0.085531,0.9775


In [ ]:
# 손실함수, 정확도 그래프 그리기
